# Multivariate Kernel Density Estimator

This example compares the statsmodel multivariate KDE with a ROC GPU version

In [1]:
%cd -q ../..

In [2]:
import numpy as np
from timeit import default_timer as timer
# Import for statsmodels KDE
import statsmodels.api as sm
# Import Bokeh for plotting
from bokeh.plotting import figure, show, output_notebook
from bokeh.models.layouts import Column
from bokeh import palettes
# Import our custom ROC KDE implementation
from numba_roc_examples.kerneldensity.roc_imp import approx_bandwidth, build_support_nd, roc_multi_kde, calc_rms
from numba_roc_examples.kerneldensity.plotting import RGBColorMapper

In [3]:
output_notebook()

Loading BokehJS ...

## Prepare sample data

In [4]:
size = 200   # Configurable for different sample size
samples = np.squeeze(np.dstack([np.random.normal(size=size),
                                np.random.normal(size=size)]))
bwlist = [approx_bandwidth(samples[:, k])
          for k in range(samples.shape[1])]
bandwidths = np.array(bwlist)
support = build_support_nd(samples, bandwidths)

print("Samples byte size:", samples.nbytes)
print("Support byte size:", support.nbytes)

Samples byte size: 3200
Support byte size: 640000


## Run Statsmodels KDE

In [5]:
kde = sm.nonparametric.KDEMultivariate(samples, var_type='cc', bw=bwlist)
pdf_sm = kde.pdf(support)

### Plot

Plot the density

In [6]:
N = int(np.sqrt(pdf_sm.size))
normed = pdf_sm/np.ptp(pdf_sm)
cm = RGBColorMapper(0, 1, palettes.Spectral11)
img = cm.color_rgba(normed).view(dtype=np.uint32).reshape(N, N)

x0 = support[0, 0]
y0 = support[0, 1]
x1 = support[-1, 0]
y1 = support[-1, 1]
dw = x1 - x0
dh = y1 - y0

fig_sm = figure(title='KDE-statsmodel', x_range=[x0, x1], y_range=[y0, y1])
fig_sm.image_rgba(image=[img], x=[x0], y=[y0], dw=[dw], dh=[dh])
show(fig_sm)

## Run ROC GPU KDE

In [7]:
pdf_roc = np.zeros(support.shape[0], dtype=np.float64)
roc_multi_kde(support, samples, bandwidths, pdf_roc)

### Plot
Plot the density

In [8]:
N = int(np.sqrt(pdf_roc.size))
normed = pdf_roc/np.ptp(pdf_roc)
cm = RGBColorMapper(0, 1, palettes.Spectral11)
img = cm.color_rgba(normed).view(dtype=np.uint32).reshape(N, N)

x0 = support[0, 0]
y0 = support[0, 1]
x1 = support[-1, 0]
y1 = support[-1, 1]
dw = x1 - x0
dh = y1 - y0

fig_roc = figure(title='KDE-numba-roc', x_range=[x0, x1], y_range=[y0, y1])
fig_roc.image_rgba(image=[img], x=[x0], y=[y0], dw=[dw], dh=[dh])
show(fig_roc)

## Benchmark

Test the two version for different size of samples.

The `driver()` function will run both versions of KDE over the same random data and support.  The results are compared
and the execution time of each version is returned.

In [9]:
def driver(size):
    # Prepare samples
    samples = np.squeeze(np.dstack([np.random.normal(size=size),
                                    np.random.normal(size=size)]))
    bwlist = [approx_bandwidth(samples[:, k])
              for k in range(samples.shape[1])]
    bandwidths = np.array(bwlist)
    support = build_support_nd(samples, bandwidths)
    # Statsmodel
    kde = sm.nonparametric.KDEMultivariate(samples, var_type='cc', bw=bwlist)
    ts = timer()
    pdf_sm = kde.pdf(support)
    time_sm = timer() - ts
    # ROC GPU
    pdf_roc = np.zeros(support.shape[0], dtype=np.float64)
    ts = timer()
    roc_multi_kde(support, samples, bandwidths, pdf_roc)
    time_roc = timer() - ts
    # Check error
    assert calc_rms(pdf_sm, pdf_roc, norm=True) < 1e-4
    return time_sm, time_roc

We will run `driver()` over 5 differents sizes to measure how each implementation scale.

In [10]:
sample_sizes = [100, 200, 300, 400, 500]
sm_timings = []
roc_timings = []
for sz in sample_sizes:
    time_sm, time_roc = driver(sz)
    sm_timings.append(time_sm)
    roc_timings.append(time_roc)

The following plots the measured data.

In [11]:
fig = figure(title='Execution Time')
fig.xaxis.axis_label = 'sample size'
fig.yaxis.axis_label = 'execution time (seconds)'
fig.line(sample_sizes, sm_timings, color='blue', legend='statsmodels')
fig.line(sample_sizes, roc_timings, color='red', legend='ROC-GPU')

fig2 = figure(title='Speedup')
fig2.xaxis.axis_label = 'sample size'
fig2.yaxis.axis_label = 'speedup (ROC over Statsmodels)'
fig2.line(sample_sizes, np.array(sm_timings)/np.array(roc_timings))

show(Column(fig, fig2))